In [15]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import datetime

import psd_tool
import data_loader
from field_models import model
import matplotlib.pyplot as plt
import spacepy
import numpy as np

import importlib
importlib.reload(psd_tool)
importlib.reload(data_loader)


%matplotlib qt

In [16]:
start = datetime.datetime(year = 2013,
                            month = 10,
                            day = 1)

end = datetime.datetime(year = 2013, 
                        month = 10, 
                        day = 4)

satellite = "A"
chosen_mu = 3000
chosen_k = 0.05
model = model.TS04D

dependencies = data_loader.load_psd_dependencies(satellite=satellite, field_model=model, start=start, end=end)

epoch, Lstar, PSD, in_out, orbit_number = psd_tool.calculate_psd(dependencies=dependencies, chosen_mu=chosen_mu, chosen_k=chosen_k, debug_mode=False, verbose=False)


Loading : PSD_DEPENDENCIES_201310_A_TS04D.npz
Time taken to interpolate over time: 0.09342169761657715
Time taken for loop: 0.38309550285339355


In [17]:
def plot_psd(LSTAR, PSD, LSTAR_MIN, LSTAR_MAX, IS_INBOUND: bool, start_of_orbit: datetime.datetime):
    
    NOT_NAN_OR_ZERO = np.isfinite(PSD) & (PSD > 0)
    PSD = PSD[NOT_NAN_OR_ZERO]
    LSTAR = LSTAR[NOT_NAN_OR_ZERO]
    
    year = str(start_of_orbit.year)
    month = str(start_of_orbit.month)
    day = str(start_of_orbit.day)
    if len(day) < 2:
        day = f"0{day}"
    hour = str(start_of_orbit.hour)
    if len(hour) < 2:
        hour = f"0{hour}"
    minute = str(start_of_orbit.minute)
    if len(minute) < 2:
        minute = f"0{minute}"
        
    PSD_to_plot = []
    Lstar_to_plot = []
    
    curr = LSTAR_MIN
    dt = 0.20
    while curr < LSTAR_MAX:
        
        bin = (curr < LSTAR) & (LSTAR < curr + dt)
        if (np.sum(bin) != 0):
            Lstar_to_plot.append(curr)
            PSD_to_plot.append(np.nanmean(PSD[bin]))
        curr += dt
    
    PSD_to_plot = np.array(PSD_to_plot)
    Lstar_to_plot = np.array(Lstar_to_plot)
    
    if len(Lstar_to_plot) > 1:
        max_distance = np.max(np.array(Lstar_to_plot[1:]) - np.array(Lstar_to_plot[0:-1]))
    else:
        max_distance = 0.0
    
    if IS_INBOUND and (max_distance < 2.0):
        plt.semilogy(Lstar_to_plot[np.isfinite(PSD_to_plot)], PSD_to_plot[np.isfinite(PSD_to_plot)], marker="*", label=f"{month}/{day}/{year} {hour}:{minute}")
    elif IS_INBOUND and (max_distance > 2.0):
        plt.semilogy(Lstar_to_plot[np.isfinite(PSD_to_plot)], PSD_to_plot[np.isfinite(PSD_to_plot)], ".", marker="*", label=f"{month}/{day}/{year} {hour}:{minute}")
    elif (not IS_INBOUND) and (max_distance < 2.0):
        plt.semilogy(Lstar_to_plot[np.isfinite(PSD_to_plot)], PSD_to_plot[np.isfinite(PSD_to_plot)], marker=".", label=f"{month}/{day}/{year} {hour}:{minute}")
    else:
        plt.semilogy(Lstar_to_plot[np.isfinite(PSD_to_plot)], PSD_to_plot[np.isfinite(PSD_to_plot)], ".", marker=".", label=f"{month}/{day}/{year} {hour}:{minute}")


In [18]:
min_psd = 1e-12
max_psd = 1e-6
min_lstar = 2.0
max_lstar = 6.0

not_nan = np.isfinite(PSD)
Lstar = Lstar[not_nan]
PSD = PSD[not_nan]
in_out = in_out[not_nan]
orbit_number = orbit_number[not_nan]

unique_orbits = np.unique(orbit_number)
start_of_orbits = []

fig, ax = plt.subplots(1,1)

for orbit in unique_orbits:
    
    inbound = (orbit_number == orbit) & (in_out == -1)
    outbound = (orbit_number == orbit) & (in_out == 1)
    
    if np.any(np.nonzero(inbound)[0]):
        
        start_of_inbound_orbit = epoch[np.nonzero(inbound)[0][0]]
        
        plot_psd(Lstar[inbound], PSD[inbound], min_lstar, max_lstar, True, start_of_inbound_orbit)
        start_of_orbits.append(start_of_inbound_orbit)

    if np.any(np.nonzero(outbound)[0]):
        
        start_of_outbound_orbit = epoch[np.nonzero(outbound)[0][0]]
        
        plot_psd(Lstar[outbound], PSD[outbound], min_lstar, max_lstar, True, start_of_outbound_orbit)
        start_of_orbits.append(start_of_outbound_orbit)

ax.set_axisbelow(True)
ax.yaxis.grid(color='gray', linestyle='dashed')
ax.xaxis.grid(color='gray', linestyle='dashed')
plt.xlabel("L*")
plt.ylabel("PSD $(c/MeV/cm)^{3}$")

plt.ylim((min_psd, max_psd))
plt.xlim((min_lstar, max_lstar))

handles, labels = plt.gca().get_legend_handles_labels()
order = np.argsort(start_of_orbits)
colors = plt.get_cmap("viridis", len(order)) # Initialize holder for trajectories
j = 0
for i in order:
    handles[i].set_color(colors(j))
    j += 1

plt.legend([handles[i] for i in order], [labels[i] for i in order])
plt.title(rf"RBSP-{satellite.upper()}, $\mu$ = {chosen_mu} (MeV / G), K = {chosen_k} ($R_E$$G^{{1/2}}$), {model.name}")
fig.set_size_inches(12, 8)
#plt.savefig((OUTPUT_DIR + f"{year}-{month}-{day}-RBSP-{sat.upper()}-Mu-{int(CHOSEN_MU)}-K-{int(CHOSEN_K * 100)}-{FIELD_MODEL.name}.pdf"), format="pdf", dpi=100)

plt.show()